The VideoFrameGenerator I created can be installed via pip command:

In [ ]:
%cd /content/
import os
os.mkdir("Dataset") 
os.mkdir("Testset")

In [ ]:
# Download
#dataset
%cd /content/Dataset
#!curl -L "Link for rar file here" > Dataset.rar; unrar x Dataset.rar; rm Dataset.rar  

#testset
%cd /content/Testset
#!curl -L "Link for rar file here" > Testset.rar; unrar x Testset.rar; rm Testset.rar  



In [ ]:
%cd /content
!rm -r chkp/
!rm -r logs/

In [ ]:
!pip install keras-video-generators



In [ ]:
%cd /content/
import os
import glob
import keras
from keras_video import VideoFrameGenerator
# use sub directories names as classes
classes = [i.split(os.path.sep)[1] for i in glob.glob('Dataset/*')]
classes.sort()
# some global params
SIZE = (112, 112)
CHANNELS = 3
NBFRAME = 10
BS = 8
# pattern to get videos and classes
glob_pattern='Dataset/{classname}/*.mp4'
# for data augmentation
data_aug = keras.preprocessing.image.ImageDataGenerator(
    zoom_range=.1,
    horizontal_flip=True,
    rotation_range=8,
    width_shift_range=.2,
    height_shift_range=.2)
# Create video frame generator
train = VideoFrameGenerator(
    classes=classes, 
    glob_pattern=glob_pattern,
    nb_frames=NBFRAME,
    split_val=.2,
    shuffle=True,
    batch_size=BS,
    target_shape=SIZE,
    nb_channel=CHANNELS,
    transformation=data_aug,
    use_frame_cache=False)

import keras_video.utils
valid = train.get_validation_generator()

To get the validation data:

Here are samples from train generator — you can use:

In [ ]:
import keras_video.utils
keras_video.utils.show_sample(train)

So, we need to create that ConvNet. I recommend creating a function that returns the model. Here, I’m using a simple model. I made several convolutions with batch normalization.

In [ ]:
from keras.layers import Conv2D, BatchNormalization, \
    MaxPool2D, GlobalMaxPool2D
def build_convnet(shape=(112, 112, 3)):
    momentum = .9
    model = keras.Sequential()
    model.add(Conv2D(64, (3,3), input_shape=shape,
        padding='same', activation='relu'))
    model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    
    model.add(MaxPool2D())
    
    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    
    model.add(MaxPool2D())
    
    model.add(Conv2D(256, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(256, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    
    model.add(MaxPool2D())
    
    model.add(Conv2D(512, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(512, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    
    # flatten...
    model.add(GlobalMaxPool2D())
    return model

OK, it’s now time to build the time distributed model. I was using LSTM that is very efficient, but I finally use GRU that is conceptually the same but using less memory, but the cost is that it is less efficient. But one more time, we’re testing…

In [ ]:
from keras.layers import TimeDistributed, GRU, Dense, Dropout, LSTM
def action_model(shape=(NBFRAME, 112, 112, CHANNELS), nbout=3):
    # Create our convnet with (112, 112, 3) input shape
    convnet = build_convnet(shape[1:])
    convnet.summary()
    
    # then create our final model
    model = keras.Sequential()
    # add the convnet with (5, 112, 112, 3) shape
    model.add(TimeDistributed(convnet, input_shape=shape))
    # here, you can also use GRU or LSTM
    model.add(GRU(64))
    # and finally, we make a decision network
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(nbout, activation='softmax'))
    return model

 

Because the dataset is not very large, I recommend using Dropout to avoid overfitting (it will happen anyway… but at least we reduce the problem)
Let’s create the model:

In [ ]:
INSHAPE=(NBFRAME,) + SIZE + (CHANNELS,) # (5, 112, 112, 3)
model = action_model(INSHAPE, len(classes))
optimizer = keras.optimizers.Adam(0.001)
model.compile(
    optimizer,
    'categorical_crossentropy',
    metrics=['acc']
)
model.summary()

# #Training





In [ ]:
EPOCHS=200
# create a "chkp" directory before to run that
# because ModelCheckpoint will write models inside
#keras.callbacks.EarlyStopping(monitor="acc", patience=10),
callbacks = [
    keras.callbacks.ReduceLROnPlateau(verbose=1),
    keras.callbacks.ModelCheckpoint(
        'chkp/weights.{epoch:02d}-{val_loss:.2f}.hdf5',
        verbose=1),
        keras.callbacks.TensorBoard(log_dir='logs')
]
model.fit(
    train,
    validation_data=valid,
    verbose=1,
    epochs=EPOCHS,
    callbacks=callbacks
)

In [ ]:
%cd /content
%load_ext tensorboard
%tensorboard --logdir logs
%reload_ext tensorboard

#   Testing

create testset 

In [ ]:
%cd /content/
import os
import glob
import keras
from keras_video import VideoFrameGenerator
# use sub directories names as classes
classes = [i.split(os.path.sep)[1] for i in glob.glob('Testset/*')]
classes.sort()
# some global params
SIZE = (112, 112)
CHANNELS = 3
NBFRAME = 10
BS = 8
# pattern to get videos and classes
glob_pattern='Testset/{classname}/*.mp4'
# for data augmentation
data_aug = keras.preprocessing.image.ImageDataGenerator(
    zoom_range=.1,
    horizontal_flip=True,
    rotation_range=8,
    width_shift_range=.2,
    height_shift_range=.2)
# Create video frame generator
test_gen = VideoFrameGenerator(
    classes=classes, 
    glob_pattern=glob_pattern,
    nb_frames=NBFRAME,
    shuffle=False,
    split_val=.99,
    batch_size=BS,
    target_shape=SIZE,
    nb_channel=CHANNELS,
    transformation=data_aug,
    use_frame_cache=False)

import keras_video.utils
test = test_gen.get_validation_generator()

custom show_result method

In [ ]:
"""
Utils module to provide some nice functions to develop with Keras and
Video sequences.
"""

import numpy as np
import random as rnd
import matplotlib.pyplot as plt


def custom_show_sample(pred, g, index=0, random=False, row_width=22, row_height=5):
    """ Displays a batch using matplotlib.

    params:

    - g: keras video generator
    - index: integer index of batch to see (overriden if random is True)
    - random: boolean, if True, take a random batch from the generator
    - row_width: integer to give the figure height
    - row_height: integer that represents one line of image, it is multiplied by \
    the number of sample in batch.
    """
    total = len(g)
    sample = index
    batch_offset = BS*(index)

    assert index < len(g)
    sample = g[sample]
    sequences = sample[0]
    labels = sample[1]
    classes = ['Approach', 'ChangeLane', 'Follow', 'FreeRide']
    rows = len(sequences)
    index = 1
    fig = plt.figure(figsize=(row_width, row_height*rows))
    for batchid, sequence in enumerate(sequences):
        classid = np.argmax(labels[batchid])
        classname = g.classes[classid]
        cols = len(sequence)
        pred_acc = np.argmax(pred, axis=1)
        pred_index = pred_acc[batchid + batch_offset]
        for image in sequence:
            plt.subplot(rows, cols, index)
            plt.title(classname)
            plt.imshow(image)
            plt.annotate(classes[pred_index], xy=(0, 100))
            acc = pred[batchid + batch_offset]
            acc_str = []
            for i in range(len(acc)):
              k = "{:.2f}".format(acc[i])
              acc_str.append(float(k))
            plt.annotate(acc_str, xy=(30, 90))
            plt.axis('off')
            index += 1
    plt.show()


custom method for printing results

In [ ]:
def print_results(results):
  #print(results)
  result_class= np.argmax(results, axis=1)
  print(result_class)

  #print confidenc of each prediction
  seq = -1
  seq_batch = -1
  batch = 0
  classes = ['Approach', 'ChangeLane', 'Follow', 'FreeRide']


  for conf in results:
    seq = seq + 1
    seq_batch = seq_batch + 1
    if (seq % (BS ) == 0 and seq != 0):
      seq_batch = seq_batch - (BS )
    if (seq % (BS ) == 0 and seq != 0):
      batch = batch + 1
    #print('seq',seq)
    #print('seq_batch',seq_batch)
    #print('batch',batch)
    acc_str = []
    for i in range(len(conf)):
      k = "{:.2f}".format(conf[i])
      acc_str.append(float(k))
    print('sequence ',seq )
    classid = np.argmax(test[batch][1][seq_batch -1])
    print('correct class:', classes[classid], ' - ', 'prediction:', classes[result_class[seq]])
    print('confidence: ', acc_str)

Accuracy e visual output

In [ ]:
import numpy as np
#compute and print average accuracy
accuracy = model.evaluate(test)
#compute and print predictions
results = model.predict(test)

#print results
print_results(results)
  
#show results
batch = len(test)
for i in range(batch):
  custom_show_sample(results, test,i)


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
#Copy chkp to GDrive
#!cp "/content/chkp/weights.150-0.53.hdf5" -r "/content/gdrive/My Drive/"
#!cp "/content/chkp/weights.140-0.53.hdf5" -r "/content/gdrive/My Drive/"

#Copy logs to GDrive
#!cp "/content/gdrive/MyDrive/100%/day_cut_20frame_full_95%/logs" -r "/content/logs"